# Image Classification using Keras

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/soumik12345/wandb-addons/blob/main/docs/keras/examples/image_classififcation.ipynb)

This notebook demonstrates how to perform transfer-learning using a pre-trained model from Keras Applications and perform experiment tracking using Weights & Biases.

## Install Dependencies

In [ ]:
!pip install -q namex
!apt install python3.10-venv
!git clone https://github.com/keras-team/keras-core.git && cd keras-core
!python pip_build.py --install
!pip install -q git+https://github.com/keras-team/keras-cv
!pip install --upgrade -q git+https://github.com/soumik12345/wandb-addons

## Import the Libraries

In [ ]:
import os

# works with tensorflow, jax and torch backends
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import tensorflow.image as tf_image
import tensorflow.data as tf_data
import tensorflow_datasets as tfds

import keras_core as keras
from keras_core import ops
from keras_core import layers
from keras_core import applications

import wandb
from wandb_addons.keras import WandbMetricsLogger
from wandb_addons.keras import WandbModelCheckpoint
from wandb_addons.keras import WandBImageClassificationCallback

## Initialize a Weights & Biases run and Set up the Configs

In [ ]:
wandb.init(project="keras-community-days", job_type="classification")

config = wandb.config
config.batch_size = 64
config.dataset_name = "stanford_dogs"
config.image_size = 224
config.epochs = 10
config.top_dropout_rate = 0.2

## Setting up the Dataset Pipeline

### Load the data

In [ ]:
(train_dataset, test_dataset), info = tfds.load(
    config.dataset_name,
    split=["train", "test"],
    with_info=True,
    as_supervised=True
)
train_dataset = train_dataset.take(100)
test_dataset = test_dataset.take(100)
config.classes = [name.split("-")[-1] for name in info.features["label"].names]

### Resize the Data

In [ ]:
train_dataset = train_dataset.map(
    lambda image, label: (
        tf_image.resize(image, (config.image_size, config.image_size)), label
    )
)
test_dataset = test_dataset.map(
    lambda image, label: (
        tf_image.resize(image, (config.image_size, config.image_size)), label
    )
)

### Preprocess the Labels

In [ ]:
def input_preprocess(image, label):
    label = tf.one_hot(label, len(config.classes))
    return image, label


train_dataset = train_dataset.map(
    input_preprocess, num_parallel_calls=tf_data.AUTOTUNE
)
train_dataset = train_dataset.batch(
    batch_size=config.batch_size, drop_remainder=True
)
train_dataset = train_dataset.prefetch(tf_data.AUTOTUNE)

test_dataset = test_dataset.map(
    input_preprocess, num_parallel_calls=tf_data.AUTOTUNE
)
test_dataset = test_dataset.batch(
    batch_size=config.batch_size, drop_remainder=True
)
test_dataset = test_dataset.prefetch(tf_data.AUTOTUNE)

## Defining the Model

In [ ]:
inputs = keras.Input(shape=(config.image_size, config.image_size, 3))
x = inputs
model = applications.EfficientNetB0(
    include_top=False, input_tensor=x, weights="imagenet"
)

# Freeze the pretrained weights
model.trainable = False

# Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

x = layers.Dropout(config.top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(len(config.classes), activation="softmax", name="pred")(x)

# Compile
model = keras.Model(inputs, outputs, name="EfficientNet")
optimizer = keras.optimizers.Adam(learning_rate=1e-2)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

## Start Training

In [ ]:
callbacks = [
    # Log your metrics as you train
    WandbMetricsLogger(log_freq="batch"),
    # Save and version your model checkpoints
    WandbModelCheckpoint("model.keras"),
    # Visualize your model's performance
    WandBImageClassificationCallback(
        dataset=test_dataset,
        class_labels=config.classes,
        max_items_for_visualization=50,
    )
]

# Start training...
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=config.epochs,
    callbacks=callbacks
)

# Finish the experiment
wandb.finish()